Structure similarity investigation of GDB-13 fragrance-like molecules database 
Input molecule as SMILES, sort by structural similarity 
Play around with fingerprint representation 
Generate structural similarity maps 

In [9]:
# first attempt - fingerprint similarity comparison, no 3D structure 
import rdkit
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.DataStructs import cDataStructs


AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [3]:
# convert SMI to csv - use pandas to convert text file to dataframe 
column_name=["Smiles"]
df = pd.read_csv('/home/martyn0000/fragrance_investigation/GDB-13-FL.smi',
                sep='\r\n', engine='python', names=column_name
                )
df = df.iloc[1: , :]
print(df.head(2))

df1 = df.iloc[:1000000,:]
print(df1.head(2))

df2 = df.iloc[30000000:,:]
print(df2.head(2))

  Smiles
1    C#C
2  C1CC1
  Smiles
1    C#C
2  C1CC1
                            Smiles
30000001   CC1CC2(CC1OCO2)C(=C)C#C
30000002  CC(C=C)C12CC(C)C(C1)OCO2


In [6]:
# sanitise input - remove S, Cl, N using pandas drop 
df1M = pd.read_csv('/home/martyn0000/fragrance_investigation/df1_1M_with_fingerprints.csv')
df1M_sanitised = df1M.copy()
regex_expression1 = "[SN]"
filter = df1M_sanitised['Smiles'.str.contains(regex_expression1)]
regex_expression2 = "(Cl)"
df1M_sanitised=df1M_sanitised[~filter]
print(df1M_sanitised.head(20))

NameError: name 'pd' is not defined

In [4]:
# rdkit calculate features - Morgan fingerprint 
 
PandasTools.AddMoleculeColumnToFrame(df1,'Smiles','molecule',includeFingerprints=True)
df1['morgan'] = df1['molecule'].map(lambda x:GetMorganFingerprintAsBitVect(x,2)) 
print(df1.head(2))

/home/martyn0000/miniconda3/envs/py310_aisem2023/lib/python3.10/site-packages/rdkit/Chem/PandasTools.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[molCol] = frame[smilesCol].map(


  Smiles                                          molecule   
1    C#C  <rdkit.Chem.rdchem.Mol object at 0x7fb83ae35ea0>  \
2  C1CC1  <rdkit.Chem.rdchem.Mol object at 0x7fb586337a60>   

                                              morgan  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  


/tmp/ipykernel_9701/274753435.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['morgan'] = df1['molecule'].map(lambda x:GetMorganFingerprintAsBitVect(x,2))


In [5]:
df1.to_csv('df1_1M_with_fingerprints.csv')

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df1,'Smiles','molecule',includeFingerprints=True)
df2['morgan'] = df2['molecule'].map(lambda x:GetMorganFingerprintAsBitVect(x,2)) 
print(df2.head(2))

In [7]:
# similarity search of database 

import rdkit.DataStructs
similarity_target=Chem.MolFromSmiles("C[C@@]1(C(C)=O)CC=C2CCCC(C)(C)[C@]2(C)[C@H]1C") 
def get_dataframe_with_x_most_similar_compounds_to_query(query, mol_df, molCol='molecule', x=int):
    query_fp = GetMorganFingerprintAsBitVect(query,2)
    mol_df['similarity'] = mol_df['morgan'].map(lambda x:DataStructs.TanimotoSimilarity(query_fp, x))
    mol_df.sort_values(['similarity'], ascending=False, inplace=True)
    return mol_df[:x].copy()  
similarity_df=get_dataframe_with_x_most_similar_compounds_to_query(similarity_target, df1, x=50)
similarity_df.head()

NameError: name 'cDataStructs' is not defined

In [ ]:
# generate similarity maps of most suitable candidates 
    # get smiles of similar molecules as a list, loop through list making  
from rdkit.Chem import Draw 
from rdkit.Chem.Draw import SimilarityMaps
smiles=similarity_df["morgan"].to_list()
for similar_structure in list: 
    fp = SimilarityMaps.GetMorganFingerprint(similar_structure, fpType='bv')
    fig, maxweight = SimilarityMaps.GetSimilarityMapForFingerprint(similarity_target, similar_structure, SimilarityMaps.GetMorganFingerprint)